In [54]:
import requests
import altair as alt
import pandas as pd
%pip install streamlit
%pip install altair pandas vega_datasets
%pip install -U altair_viewer


Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [55]:
#pd.read_json(requests.get("https://api.census.gov/data/2019/pep/charagegroups?get=NAME,POP&HISP=2&for=state:*").json())

#data = {"State":['Mississippi','Missouri','Montana'],
#'Population':['100110','268708','43289']}


#chart = alt.Chart(df).mark_bar().encode(
 #   x='State:N',
  #  y='Population:Q',
   # tooltip=['State:N', 'Population:Q']).properties(
 #   title='US Population by State',
  #  width=600,
   # height=400)
#chart.show()




In [56]:
jsonlist = requests.get("https://api.census.gov/data/2019/pep/charagegroups?get=NAME,POP&HISP=2&for=state:*").json()
states =[]
populations =[]
for list in jsonlist:
    states.append(list[0])
    populations.append(list[1])
data = {"State":states,"population":populations
}
df=pd.DataFrame(data)
df


,State,population
0,NAME,POP
1,Mississippi,100110
2,Missouri,268708
3,Montana,43289
4,Nebraska,219645
5,Nevada,900600
6,New Hampshire,54589
7,New Jersey,1856844
8,New Mexico,1032942
9,New York,3751058
